# **NLP. Tarea 2: Minería de Texto Básica**.

*Diego Moreno*

# 2. Bolsas de Palabras, Bigramas y Emociones

Importamos las librerías necesarias para la tarea:

In [207]:
import os
import re
import numpy as np
#from keras.preprocessing.text import Tokenizer 
import nltk
#import glob
from nltk import FreqDist
#from nltk.tokenize import RegexpTokenizer
#from nltk.text import Text
from nltk.tokenize import TweetTokenizer 
from nltk import ngrams
#from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score
#from nltk.corpus import stopwords

Obtenemos los datos Mex-A3t y preprocesamos:

In [504]:
def get_text_from_file(path_corpus, path_labels):
    tr_txt = []
    tr_y = []
    
    with open(path_corpus, 'r') as f_corpus, open(path_labels, 'r') as f_labels:
        for twitt in f_corpus:
            tr_txt += [twitt]
        for label in f_labels:
            tr_y += [label]
    return tr_txt, tr_y

def sort_freq_dict(fdict):
    aux = [(fdict[key], key) for key in fdict]
    aux.sort()
    aux.reverse()
    return aux

In [505]:
tr_txt, tr_y = get_text_from_file('mex_train.txt', 'mex_train_labels.txt')
tr_y = list(map(int, tr_y))
val_txt, val_y = get_text_from_file('mex_val.txt', 'mex_val_labels.txt')
val_y = list(map(int, val_y))

In [506]:
tokenizer = TweetTokenizer()

corpus_words = []
for doc in tr_txt:
    corpus_words += tokenizer.tokenize(doc)

fdist = nltk.FreqDist(corpus_words)

In [507]:
v = sort_freq_dict(fdist) 
V = v[:5000]

In [508]:
dict_ind = dict()
count = 0
for _, word in V:
    dict_ind[word] = count
    count += 1

## 2.1 Bolsas de Términos

A continuación, coloco todos los constructores de las bolsas de términos; correponden a las bolsas solicitadas para los puntos 2.1, 2.2, 2.3, 2.4, 2.5 y 2.6.

In [104]:
def build_bow_binary(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado binario.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=int)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_ind:
                bow[doc_count, dict_ind[word]] = 1
        doc_count += 1
        
    return bow

def build_bow_tf(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento. 
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=int)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_ind:
                bow[doc_count, dict_ind[word]] = fdist_doc[word]
        doc_count += 1
        
    return bow

def build_bow_tfidf(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento y penalizado por la frecuencia del término
    en cada documento.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    idf = np.zeros(len(V), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_ind:
                bow[doc_count, dict_ind[word]] = fdist_doc[word]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for word in dict_ind:
        df = sum(bow[:, dict_ind[word]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[word]] = np.log(N) - np.log(df)
        
    bow *= idf
    return bow

def build_bow_binary_norm(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado binario.
    Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_ind:
                bow[doc_count, dict_ind[word]] = 1
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        bow[i] /= sum_w
        
    return bow

def build_bow_tf_norm(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento. Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_ind:
                bow[doc_count, dict_ind[word]] = fdist_doc[word]
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        bow[i] /= sum_w
        
    return bow

def build_bow_tfidf_norm(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento y penalizado por la frecuencia del término
    en cada documento. Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    idf = np.zeros(len(V), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in dict_ind:
                bow[doc_count, dict_ind[word]] = fdist_doc[word]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for word in dict_ind:
        df = sum(bow[:, dict_ind[word]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[word]] = np.log(N) - np.log(df)
        
    bow *= idf
    
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        bow[i] /= sum_w
            
    return bow

### Evaluación de los modelos con SVM y reporte de clasificación:

## 2.1 Esquema de pesado binario:

In [100]:
bow_tr = build_bow_binary(tr_txt, V, dict_ind)
bow_val = build_bow_binary(val_txt, V, dict_ind)

**Clasificación**

In [75]:
params = {'C' : [.05, .12, .25, .5, 1, 2, 4]}

svr = svm.LinearSVC(class_weight = 'balanced')
grid = GridSearchCV(estimator = svr, param_grid = params, n_jobs = 8,
                   scoring = 'f1_macro', cv = 5)
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [82]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[331  66]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       397
           1       0.72      0.78      0.75       219

    accuracy                           0.81       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.81      0.81       616



La clasificación de un twitt agresivo tuvo un f1-score de **0.75** y el modelo en general un accuracy de **0.81**.

## 2.2 Esquema de pesado por frecuencia:

In [83]:
bow_tr = build_bow_tf(tr_txt, V, dict_ind)
bow_val = build_bow_tf(val_txt, V, dict_ind)

In [84]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [85]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[334  63]
 [ 50 169]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       397
           1       0.73      0.77      0.75       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.82      0.82       616



La clasificación de un twitt agresivo tuvo un f1-score de **0.75** y el modelo en general un accuracy de **0.82**. Mejorando así, por 0.01 el accuracy del binario.

## 2.3 Esquema de pesado por frecuencia en documento y penalizando por frecuencia en todos los documentos.

In [105]:
bow_tr = build_bow_tfidf(tr_txt, V, dict_ind)
bow_val = build_bow_tfidf(val_txt, V, dict_ind)

In [106]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [107]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[321  76]
 [ 77 142]]
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       397
           1       0.65      0.65      0.65       219

    accuracy                           0.75       616
   macro avg       0.73      0.73      0.73       616
weighted avg       0.75      0.75      0.75       616



La clasificación de un twitt agresivo tuvo un f1-score de **0.65**, empeorando por un 0.1 a lo anterior obtenido, y el modelo en general un accuracy de **0.75**. Empeorando así, por 0.07 el accuracy anterior.

## 2.4 Ahora realicemos lo anterior pero con una normalización L2, a continuación el esquema binario normalizado:

In [108]:
bow_tr = build_bow_binary_norm(tr_txt, V, dict_ind)
bow_val = build_bow_binary_norm(val_txt, V, dict_ind)

In [109]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [110]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[323  74]
 [ 48 171]]
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       397
           1       0.70      0.78      0.74       219

    accuracy                           0.80       616
   macro avg       0.78      0.80      0.79       616
weighted avg       0.81      0.80      0.80       616



La clasificación de un twitt agresivo tuvo un f1-score de **0.74**, empeorando por un 0.01 a lo anterior obtenido no normalizando, y el modelo en general un accuracy de **0.80**. Empeorando así, por 0.01 el accuracy anterior de igual manera.

## 2.5 Esquema de pesado por frecuencia normalizado:

In [111]:
bow_tr = build_bow_tf_norm(tr_txt, V, dict_ind)
bow_val = build_bow_tf_norm(val_txt, V, dict_ind)

In [112]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [113]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[322  75]
 [ 50 169]]
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       397
           1       0.69      0.77      0.73       219

    accuracy                           0.80       616
   macro avg       0.78      0.79      0.78       616
weighted avg       0.80      0.80      0.80       616



Similarmente, la clasificación de un twitt agresivo tuvo un f1-score de **0.73**, empeorando por un 0.02 a lo anterior obtenido no normalizando, y el modelo en general un accuracy de **0.80**. Empeorando así, por 0.02 el accuracy anterior al no normalizar.

## 2.6 Esquema de pesado por TF-IDF normalizado:

In [114]:
bow_tr = build_bow_tfidf_norm(tr_txt, V, dict_ind)
bow_val = build_bow_tfidf_norm(val_txt, V, dict_ind)

In [115]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [116]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[317  80]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.80      0.83       397
           1       0.68      0.78      0.72       219

    accuracy                           0.79       616
   macro avg       0.77      0.79      0.78       616
weighted avg       0.80      0.79      0.79       616



Para este caso, la clasificación de un twitt agresivo tuvo un f1-score de **0.72**, mejorando a 0.65 de lo anterior obtenido no normalizando, y el modelo en general un accuracy de **0.79**. Mejorando también, por 0.04 el accuracy anterior al no normalizar.

## 2.7 Comparación de resultados:

In [414]:
print('Pesado  | F1-score | Accuracy\n_______/_\________/_\________')
print('Binary  |   0.75   |   0.81\n--------+----------+---------')
print('Binary  |   0.74   |   0.80\nnormed  |          |\n--------+----------+---------')
print('TF      |   0.75   |   0.82\n--------+----------+---------')
print('TF      |   0.73   |   0.80\nnormed  |          |\n--------+----------+---------')
print('TF-IDF  |   0.65   |   0.75\n--------+----------+---------')
print('TF-IDF  |   0.72   |   0.79\nnormed  |          |')

Pesado  | F1-score | Accuracy
_______/_\________/_\________
Binary  |   0.75   |   0.81
--------+----------+---------
Binary  |   0.74   |   0.80
normed  |          |
--------+----------+---------
TF      |   0.75   |   0.82
--------+----------+---------
TF      |   0.73   |   0.80
normed  |          |
--------+----------+---------
TF-IDF  |   0.65   |   0.75
--------+----------+---------
TF-IDF  |   0.72   |   0.79
normed  |          |


Notamos a grandes rasgos que la normalización solamente beneficia al pesado TF-IDF, y el método con mejor desempeño en general es el TF manteniéndose constante. Por lo tanto, en el siguiente punto, elegiremos al pesado TF.

## 2.8 Evaluamos con más y menos términos para TF

Menos términos (500):

In [155]:
V = v[:500]

In [156]:
dict_ind = dict()
count = 0
for _, word in V:
    dict_ind[word] = count
    count += 1

In [157]:
bow_tr = build_bow_tf(tr_txt, V, dict_ind)
bow_val = build_bow_tf(val_txt, V, dict_ind)

In [158]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [159]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[320  77]
 [ 55 164]]
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       397
           1       0.68      0.75      0.71       219

    accuracy                           0.79       616
   macro avg       0.77      0.78      0.77       616
weighted avg       0.79      0.79      0.79       616



Más términos (10000):

In [204]:
V = v[:10000]

In [199]:
dict_ind = dict()
count = 0
for _, word in V:
    dict_ind[word] = count
    count += 1

In [200]:
bow_tr = build_bow_tf(tr_txt, V, dict_ind)
bow_val = build_bow_tf(val_txt, V, dict_ind)

In [201]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [202]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[335  62]
 [ 50 169]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       397
           1       0.73      0.77      0.75       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.82      0.82       616



In [413]:
print('Términos | F1-score | Accuracy | True Pos | True Neg\n________/_\________/_\________/_\________/_\________')
print('500      |   0.71   |   0.79   |    320   |    164\n---------+----------+----------+----------+---------')
print('5000     |   0.75   |   0.82   |    334   |    169\n---------+----------+----------+----------+---------')
print('10000    |   0.75   |   0.82   |    335   |    169')

Términos | F1-score | Accuracy | True Pos | True Neg
________/_\________/_\________/_\________/_\________
500      |   0.71   |   0.79   |    320   |    164
---------+----------+----------+----------+---------
5000     |   0.75   |   0.82   |    334   |    169
---------+----------+----------+----------+---------
10000    |   0.75   |   0.82   |    335   |    169


Podemos notar que para pocos términos, la clasificación es basada en pocos twitts por lo que no logra clasificar de manera correcta el conjunto de validación, teniendo como f-score y accuracy 71 y 79 respectivamente. Para 5000 términos, el f-score base es de 75 y accuracy de 82. Sin embargo, para más términos (10000) estos scores no aumentan significativamente, empero, al observar los verdaderos positivos, con un entrenamiento de 10000 palabras solo se aumenta a 1 twitt clasificado de manera correcta. 

## 2.9 Concatenación con bolsa de bi-gramas

In [292]:
# Obtenemos los bigramas del texto
bigrams_tr = []
for tr in tr_txt:
    bigram = ngrams(tokenizer.tokenize(tr), 2)
    bigrams_tr += bigram

# Validación
# Obtenemos los bigramas del texto
bigrams_val = []
for tr in val_txt:
    bigram = ngrams(tokenizer.tokenize(tr), 2)
    bigrams_val += bigram

# Primeros 1000
v_bigr = sort_freq_dict(nltk.FreqDist(bigrams_tr)) 
V_bigr = v_bigr[:1000]

# Directorio de índices
dict_ind_bigr = dict()
count = 0
for _,bigr in V_bigr:
    dict_ind_bigr[bigr] = count
    count += 1

In [337]:
def build_bobg_binary_norm(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de bigramas con un método de pesado binario.
    Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        bigram = ngrams(tokenizer.tokenize(tr), 2)
        fdist_doc = nltk.FreqDist(bigram)
        for bigr in fdist_doc:
            if bigr in dict_ind:
                bow[doc_count, dict_ind[bigr]] = 1
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
        
    return bow

def build_bobg_tf_norm(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de bigramas con pesado TF.
    Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        bigram = ngrams(tokenizer.tokenize(tr), 2)
        fdist_doc = nltk.FreqDist(bigram)
        for bigr in fdist_doc:
            if bigr in dict_ind:
                bow[doc_count, dict_ind[bigr]] = fdist_doc[bigr]
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
        
    return bow

def build_bobg_tfidf_norm(tr_txt, V, dict_ind):
    '''
    Constructor de la bolsa de bigramas con un método de pesado TF-IDF. 
    Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(V)), dtype=float)
    idf = np.zeros(len(V), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        bigram = ngrams(tokenizer.tokenize(tr), 2)
        fdist_doc = nltk.FreqDist(bigram)
        for bigr in fdist_doc:
            if bigr in dict_ind:
                bow[doc_count, dict_ind[bigr]] = fdist_doc[bigr]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for bigr in dict_ind:
        df = sum(bow[:, dict_ind[bigr]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[bigr]] = np.log(N) - np.log(df)
        
    bow *= idf
    
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
            
    return bow

In [338]:
# Bolsa de términos
bow_tr = build_bow_tf(tr_txt, V, dict_ind)
bow_val = build_bow_tf(val_txt, V, dict_ind)

In [339]:
# Bolsa de bigramas
bow_tr_bi = build_bobg_tfidf_norm(tr_txt, V_bigr, dict_ind_bigr)
bow_val_bi = build_bobg_tfidf_norm(val_txt, V_bigr, dict_ind_bigr)

In [340]:
# Concatenamos
bow_tr = np.concatenate((bow_tr, bow_tr_bi), axis=1)
bow_val = np.concatenate((bow_val, bow_val_bi), axis=1)

In [333]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [334]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[332  65]
 [ 52 167]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       397
           1       0.72      0.76      0.74       219

    accuracy                           0.81       616
   macro avg       0.79      0.80      0.80       616
weighted avg       0.81      0.81      0.81       616



Se intentó concatenar una bolsa de bigramas pesado con binario, TF y TF-IDF y normalizado L2, con el esquema de pesado TF para el texto normal. Sin embargo, no se encontró una mejoría ni en f1-score ni en accuracy, puesto que ahora se obtiene 0.74 y 0.81 respectivamente, cuando antes se contaba con un 0.75 y 0.82 sin los bigramas.

## 2.10 Bolsa de emociones.

La estrategia para dicha bolsa será para cada twitt, revisar cada palabra dicha en él en las palabras de emolex y ver el sentimiento de la palabra para así, contemplarlo en la columna correcta. La bolsa ahora contendrá 8 columnas correspondientes a las emociones.

In [524]:
def get_text_from_file(path_corpus):
    tr_txt = []
    tr_y = []
    
    with open(path_corpus, 'r') as f_corpus:
        for word in f_corpus:
            word = word.split()
            if len(word) < 5:
                tr_txt += [word[1]]
                tr_y += [word[2]]
            
    return tr_txt, tr_y

In [525]:
emolex, emolex_y = get_text_from_file('Spanish-es-NRC-Emotion-Intensity-Lexicon-v1.txt')
emolex = emolex[1:]
emolex_y = emolex_y[1:]

In [526]:
emolex_dict = {}
for i in range(len(emolex)):
    word = emolex[i]
    emolex_dict[word] = emolex_y[i]
    
emolex_dict_ind = {}
count = 0
for word in set(emolex_y):
    emolex_dict_ind[word] = count
    count += 1

In [527]:
def build_bow_binary_emolex(tr_txt, emo_dict, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado binario.
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=int)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = 1
        doc_count += 1
        
    return bow

def build_bow_tf_emolex(tr_txt, emo_dict, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento. 
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=int)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = fdist_doc[word]
        doc_count += 1
        
    return bow

def build_bow_tfidf_emolex(tr_txt, emo_dict, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento y penalizado por la frecuencia del término
    en cada documento.
    '''
    bow = np.zeros((len(tr_txt), len(emo_dict)), dtype=float)
    idf = np.zeros(len(emo_dict), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]]  = fdist_doc[word]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for word in emo_dict:
        emo = emo_dict[word]
        df = sum(bow[:, dict_ind[emo]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[emo]] = np.log(N) - np.log(df)
        
    bow *= idf
    return bow

def build_bow_binary_norm_emolex(tr_txt, emo_dict, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado binario.
    Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = 1
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
        
    return bow

def build_bow_tf_norm_emolex(tr_txt, emo_dict, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento. Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = fdist_doc[word]
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
        
    return bow

def build_bow_tfidf_norm_emolex(tr_txt, emo_dict, dict_ind):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento y penalizado por la frecuencia del término
    en cada documento. Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(emo_dict)), dtype=float)
    idf = np.zeros(len(emo_dict), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]]  = fdist_doc[word]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for word in emo_dict:
        emo = emo_dict[word]
        df = sum(bow[:, dict_ind[emo]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[emo]] = np.log(N) - np.log(df)
        
    bow *= idf
    
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
            
    return bow

## 2.11 Evaluamos las bolsas de emociones y comparamos

Emolex con binario:

In [400]:
bow_tr = build_bow_binary_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val = build_bow_binary_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [401]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [402]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[183 214]
 [ 83 136]]
              precision    recall  f1-score   support

           0       0.69      0.46      0.55       397
           1       0.39      0.62      0.48       219

    accuracy                           0.52       616
   macro avg       0.54      0.54      0.52       616
weighted avg       0.58      0.52      0.53       616



Emolex con TF:

In [390]:
bow_tr = build_bow_tf_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val = build_bow_tf_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [385]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [386]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[183 214]
 [ 85 134]]
              precision    recall  f1-score   support

           0       0.68      0.46      0.55       397
           1       0.39      0.61      0.47       219

    accuracy                           0.51       616
   macro avg       0.53      0.54      0.51       616
weighted avg       0.58      0.51      0.52       616



Emolex con TF-IDF:

In [387]:
bow_tr = build_bow_tfidf_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val = build_bow_tfidf_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [388]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [389]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[183 214]
 [ 85 134]]
              precision    recall  f1-score   support

           0       0.68      0.46      0.55       397
           1       0.39      0.61      0.47       219

    accuracy                           0.51       616
   macro avg       0.53      0.54      0.51       616
weighted avg       0.58      0.51      0.52       616



Emolex binario con normalizacion:

In [403]:
bow_tr = build_bow_binary_norm_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val = build_bow_binary_norm_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [404]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [405]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[259 138]
 [127  92]]
              precision    recall  f1-score   support

           0       0.67      0.65      0.66       397
           1       0.40      0.42      0.41       219

    accuracy                           0.57       616
   macro avg       0.54      0.54      0.54       616
weighted avg       0.57      0.57      0.57       616



Emolex TF con normalización:

In [406]:
bow_tr = build_bow_tf_norm_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val = build_bow_tf_norm_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [407]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [408]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[259 138]
 [127  92]]
              precision    recall  f1-score   support

           0       0.67      0.65      0.66       397
           1       0.40      0.42      0.41       219

    accuracy                           0.57       616
   macro avg       0.54      0.54      0.54       616
weighted avg       0.57      0.57      0.57       616



Emolex TF-IDF con normalización:

In [409]:
bow_tr = build_bow_tfidf_norm_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val = build_bow_tfidf_norm_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [410]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [411]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[259 138]
 [127  92]]
              precision    recall  f1-score   support

           0       0.67      0.65      0.66       397
           1       0.40      0.42      0.41       219

    accuracy                           0.57       616
   macro avg       0.54      0.54      0.54       616
weighted avg       0.57      0.57      0.57       616



In [502]:
print('Pesado  | F1-score | Accuracy\n_______/_\________/_\________')
print('Binary  |   0.48   |   0.52\n--------+----------+---------')
print('Binary  |   0.41   |   0.57\nnormed  |          |\n--------+----------+---------')
print('TF      |   0.47   |   0.51\n--------+----------+---------')
print('TF      |   0.41   |   0.57\nnormed  |          |\n--------+----------+---------')
print('TF-IDF  |   0.47   |   0.51\n--------+----------+---------')
print('TF-IDF  |   0.41   |   0.57\nnormed  |          |')

Pesado  | F1-score | Accuracy
_______/_\________/_\________
Binary  |   0.48   |   0.52
--------+----------+---------
Binary  |   0.41   |   0.57
normed  |          |
--------+----------+---------
TF      |   0.47   |   0.51
--------+----------+---------
TF      |   0.41   |   0.57
normed  |          |
--------+----------+---------
TF-IDF  |   0.47   |   0.51
--------+----------+---------
TF-IDF  |   0.41   |   0.57
normed  |          |


Notamos que el desempleño de esta estrategia no es muy bueno. Ayuda un poco el normalizar los pesos pues logra un accuracy de 0.57, sin embargo, los f1-score son bajos lo cual nos habla que puede haber más errores que aciertos en los casos que se contemplan con recall o precision.

# 3. Recurso Línguistico de Emociones Mexicano

## 3.1.1. Bolsa de emociones con el nuevo recurso:

In [517]:
def get_text_from_file(path_corpus):
    tr_txt = []
    tr_y = []
    tr_p = []
    
    with open(path_corpus, 'r', encoding='latin1') as f_corpus:
        for word in f_corpus:
            word = word.split()
            tr_txt += [word[0]]
            tr_p += [word[1]]
            tr_y += [word[2]]
            
    return tr_txt, tr_y, tr_p

In [518]:
sel, sel_y, pfa = get_text_from_file('SEL.txt')
sel = sel[1:]
sel_y = sel_y[1:]
pfa = pfa[1:]
for i in range(len(pfa)):
    pfa[i] = float(pfa[i]) 

In [519]:
sel_dict = {}
sel_dict_pfa = {}
for i in range(len(sel)):
    word = sel[i]
    sel_dict[word] = sel_y[i]
    sel_dict_pfa[word] = pfa[i]
    
sel_dict_ind = {}
count = 0
for word in set(sel_y):
    sel_dict_ind[word] = count
    count += 1

Realizamos el esquema de pesado TF normalizado:

In [476]:
bow_tr = build_bow_tf_norm_emolex(tr_txt, sel_dict, sel_dict_ind)
bow_val = build_bow_tf_norm_emolex(val_txt, sel_dict, sel_dict_ind)

In [477]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [478]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[348  49]
 [176  43]]
              precision    recall  f1-score   support

           0       0.66      0.88      0.76       397
           1       0.47      0.20      0.28       219

    accuracy                           0.63       616
   macro avg       0.57      0.54      0.52       616
weighted avg       0.59      0.63      0.59       616



Con SEL se obtiene un mejor accuracy de 0.63, veamos ahora que pasa al incorporar la probabilidad.

## 3.1.2. Incorporamos el factor de probabilidad de la afectividad:

Creamos nuevas funciones de la bolsa de emocines para incorporar dicha probabilidad:

In [479]:
def build_bow_binary_sel(tr_txt, emo_dict, dict_ind, dict_pfa):
    '''
    Constructor de la bolsa de términos con un método de pesado binario.
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = dict_pfa[word]
        doc_count += 1
        
    return bow

def build_bow_tf_sel(tr_txt, emo_dict, dict_ind, dict_pfa):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento. 
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = fdist_doc[word]*dict_pfa[word]
        doc_count += 1
        
    return bow

def build_bow_tfidf_sel(tr_txt, emo_dict, dict_ind, dict_pfa):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento y penalizado por la frecuencia del término
    en cada documento.
    '''
    bow = np.zeros((len(tr_txt), len(emo_dict)), dtype=float)
    idf = np.zeros(len(emo_dict), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]]  = fdist_doc[word]*dict_pfa[word]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for word in emo_dict:
        emo = emo_dict[word]
        df = sum(bow[:, dict_ind[emo]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[emo]] = np.log(N) - np.log(df)
        
    bow *= idf
    return bow

def build_bow_binary_norm_sel(tr_txt, emo_dict, dict_ind, dict_pfa):
    '''
    Constructor de la bolsa de términos con un método de pesado binario.
    Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = dict_pfa[word]
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
        
    return bow

def build_bow_tf_norm_sel(tr_txt, emo_dict, dict_ind, dict_pfa):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento. Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(dict_ind)), dtype=float)
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]] = fdist_doc[word]*dict_pfa[word]
        doc_count += 1
        
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
        
    return bow

def build_bow_tfidf_norm_sel(tr_txt, emo_dict, dict_ind, dict_pfa):
    '''
    Constructor de la bolsa de términos con un método de pesado por frecuencia
    de los términos en cada documento y penalizado por la frecuencia del término
    en cada documento. Además, normalizado con la norma L2.
    '''
    bow = np.zeros((len(tr_txt), len(emo_dict)), dtype=float)
    idf = np.zeros(len(emo_dict), dtype=float)
    
    # Parte de TF
    doc_count = 0
    for tr in tr_txt:
        fdist_doc = nltk.FreqDist(tokenizer.tokenize(tr))
        for word in fdist_doc:
            if word in emo_dict:
                emo = emo_dict[word]
                bow[doc_count, dict_ind[emo]]  = fdist_doc[word]*dict_pfa[word]
        doc_count += 1
        
    # Parte de IDF
    N = len(tr_txt)
    for word in emo_dict:
        emo = emo_dict[word]
        df = sum(bow[:, dict_ind[emo]] > 0)
        if df == 0:
            df = 1
        idf[dict_ind[emo]] = np.log(N) - np.log(df)
        
    bow *= idf
    
    # Normalización L2
    for i in range(len(bow)):
        sum_w = np.sqrt(bow[i]@bow[i])
        if sum_w != 0:
            bow[i] /= sum_w
            
    return bow

In [483]:
bow_tr = build_bow_binary_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val = build_bow_binary_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

In [484]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [485]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[348  49]
 [175  44]]
              precision    recall  f1-score   support

           0       0.67      0.88      0.76       397
           1       0.47      0.20      0.28       219

    accuracy                           0.64       616
   macro avg       0.57      0.54      0.52       616
weighted avg       0.60      0.64      0.59       616



In [486]:
bow_tr = build_bow_tf_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val = build_bow_tf_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

In [487]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [488]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[348  49]
 [175  44]]
              precision    recall  f1-score   support

           0       0.67      0.88      0.76       397
           1       0.47      0.20      0.28       219

    accuracy                           0.64       616
   macro avg       0.57      0.54      0.52       616
weighted avg       0.60      0.64      0.59       616



In [489]:
bow_tr = build_bow_tfidf_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val = build_bow_tfidf_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

In [490]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [491]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[351  46]
 [178  41]]
              precision    recall  f1-score   support

           0       0.66      0.88      0.76       397
           1       0.47      0.19      0.27       219

    accuracy                           0.64       616
   macro avg       0.57      0.54      0.51       616
weighted avg       0.60      0.64      0.58       616



In [492]:
bow_tr = build_bow_binary_norm_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val = build_bow_binary_norm_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

In [493]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [494]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[347  50]
 [174  45]]
              precision    recall  f1-score   support

           0       0.67      0.87      0.76       397
           1       0.47      0.21      0.29       219

    accuracy                           0.64       616
   macro avg       0.57      0.54      0.52       616
weighted avg       0.60      0.64      0.59       616



In [495]:
bow_tr = build_bow_tf_norm_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val = build_bow_tf_norm_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

In [496]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [497]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[347  50]
 [175  44]]
              precision    recall  f1-score   support

           0       0.66      0.87      0.76       397
           1       0.47      0.20      0.28       219

    accuracy                           0.63       616
   macro avg       0.57      0.54      0.52       616
weighted avg       0.59      0.63      0.59       616



In [498]:
bow_tr = build_bow_tfidf_norm_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val = build_bow_tfidf_norm_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

In [499]:
grid.fit(bow_tr, tr_y)

GridSearchCV(cv=5, estimator=LinearSVC(class_weight='balanced'), n_jobs=8,
             param_grid={'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]},
             scoring='f1_macro')

In [500]:
y_pred = grid.predict(bow_val)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[347  50]
 [176  43]]
              precision    recall  f1-score   support

           0       0.66      0.87      0.75       397
           1       0.46      0.20      0.28       219

    accuracy                           0.63       616
   macro avg       0.56      0.54      0.51       616
weighted avg       0.59      0.63      0.58       616



In [503]:
print('Pesado  | F1-score | Accuracy\n_______/_\________/_\________')
print('Binary  |   0.28   |   0.64\n--------+----------+---------')
print('Binary  |   0.29   |   0.64\nnormed  |          |\n--------+----------+---------')
print('TF      |   0.28   |   0.64\n--------+----------+---------')
print('TF      |   0.28   |   0.63\nnormed  |          |\n--------+----------+---------')
print('TF-IDF  |   0.27   |   0.64\n--------+----------+---------')
print('TF-IDF  |   0.28   |   0.63\nnormed  |          |')

Pesado  | F1-score | Accuracy
_______/_\________/_\________
Binary  |   0.28   |   0.64
--------+----------+---------
Binary  |   0.29   |   0.64
normed  |          |
--------+----------+---------
TF      |   0.28   |   0.64
--------+----------+---------
TF      |   0.28   |   0.63
normed  |          |
--------+----------+---------
TF-IDF  |   0.27   |   0.64
--------+----------+---------
TF-IDF  |   0.28   |   0.63
normed  |          |


Notamos que dicha incorporacion de probabilidad mejora a 0.64 el accuracy que se tenía, ya sea para cualquier esquema de pesado. Para la normalización, se mejora el f1-Score de la clase agresivo para el binario y empeora el accuracy para los demás esquemas.

## 3.2 Comentario sobre la estrategia de incorporación de la PFA

La idea que se realizó fue incorporar la PFA en la bolsa de emociones de la siguiente manera:
* Para el pesado binario, se cambio el 1 por la PFA de la emoción que la palabra transmite.
* Para el pesado TF, se multiplica la frecuencia por la PFA de la emoción que la palabra transmite.
* Para el pesado TF-IDF, solamente se incormpora en la parte TF como se menciona arriba.

Me pareció la más adecuada debido a que cada twitt tendrá una probabilidad de los sentimientos dependiendo de la palabra que aparece en él.

# 4. ¿Le podemos ganar a BoW con Bigramas?

## 4.1. Realizamos concatenaciones de todo lo realizado anteriormente:

Para no combinar todas las opciones posibles a lo «loco», solo combianremos los casos que los algoritmos tuvieron mejor desempeño en las evaluaciones anteriores.

Creamos las BoW con mejor desempeño:

In [564]:
# Bolsa de términos
bow_tr = build_bow_tf(tr_txt, V, dict_ind)
bow_val = build_bow_tf(val_txt, V, dict_ind)

In [565]:
# Bolsa de bigramas
bow_tr_bi = build_bobg_tf_norm(tr_txt, V_bigr, dict_ind_bigr)
bow_val_bi = build_bobg_tf_norm(val_txt, V_bigr, dict_ind_bigr)

In [566]:
# Bolsa de emociones con Emolex
bow_tr_emx = build_bow_tf_norm_emolex(tr_txt, emolex_dict, emolex_dict_ind)
bow_val_emx = build_bow_tf_norm_emolex(val_txt, emolex_dict, emolex_dict_ind)

In [567]:
# Bolsa de emociones con SEL
bow_tr_sel = build_bow_tf_norm_sel(tr_txt, sel_dict, sel_dict_ind, sel_dict_pfa)
bow_val_sel = build_bow_tf_norm_sel(val_txt, sel_dict, sel_dict_ind, sel_dict_pfa)

Mezclamos BoW con BoBigrams:

In [568]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_bi), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_bi), axis=1)

In [569]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[332  65]
 [ 52 167]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       397
           1       0.72      0.76      0.74       219

    accuracy                           0.81       616
   macro avg       0.79      0.80      0.80       616
weighted avg       0.81      0.81      0.81       616



Mezclamos BoW con BoE Emolex:

In [570]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_emx), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_emx), axis=1)

In [571]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[334  63]
 [ 51 168]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       397
           1       0.73      0.77      0.75       219

    accuracy                           0.81       616
   macro avg       0.80      0.80      0.80       616
weighted avg       0.82      0.81      0.82       616



Mezclamos BoW con BoE SEL:

In [572]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_sel), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_sel), axis=1)

In [573]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[333  64]
 [ 50 169]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       397
           1       0.73      0.77      0.75       219

    accuracy                           0.81       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.81      0.82       616



Hasta ahorita todos han dado los mismos resultados, ahora tratemos de agregar 2:

Mezclamos BoW con BoBigrams y BoE Emolex:

In [574]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_bi, bow_tr_emx), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_bi, bow_val_emx), axis=1)

In [575]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[331  66]
 [ 53 166]]
              precision    recall  f1-score   support

           0       0.86      0.83      0.85       397
           1       0.72      0.76      0.74       219

    accuracy                           0.81       616
   macro avg       0.79      0.80      0.79       616
weighted avg       0.81      0.81      0.81       616



Mezclamos BoW con BoBigrams y BoE SEL:

In [576]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_bi, bow_tr_sel), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_bi, bow_val_sel), axis=1)

In [577]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[332  65]
 [ 53 166]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       397
           1       0.72      0.76      0.74       219

    accuracy                           0.81       616
   macro avg       0.79      0.80      0.79       616
weighted avg       0.81      0.81      0.81       616



Mezclamos BoW con BoE emolex y BoE SEL:

In [578]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_emx, bow_tr_sel), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_emx, bow_val_sel), axis=1)

In [579]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[334  63]
 [ 52 167]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       397
           1       0.73      0.76      0.74       219

    accuracy                           0.81       616
   macro avg       0.80      0.80      0.80       616
weighted avg       0.82      0.81      0.81       616



Nuevamente, no tenemos resultados notables a favor. Por último intentemos las 4 características concatenadas:

In [580]:
# Concatenamos
bow_tr_con = np.concatenate((bow_tr, bow_tr_bi, bow_tr_emx, bow_tr_sel), axis=1)
bow_val_con = np.concatenate((bow_val, bow_val_bi, bow_val_emx, bow_val_sel), axis=1)

In [581]:
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[330  67]
 [ 53 166]]
              precision    recall  f1-score   support

           0       0.86      0.83      0.85       397
           1       0.71      0.76      0.73       219

    accuracy                           0.81       616
   macro avg       0.79      0.79      0.79       616
weighted avg       0.81      0.81      0.81       616



Bolsa original:

In [582]:
bow_tr_con = bow_tr
bow_val_con = bow_val
grid.fit(bow_tr_con, tr_y)
y_pred = grid.predict(bow_val_con)
print(confusion_matrix(val_y, y_pred))
print(metrics.classification_report(val_y, y_pred))

[[334  63]
 [ 50 169]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       397
           1       0.73      0.77      0.75       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.82      0.82       616



In [588]:
print('Combinación    | F1-score | Accuracy\n______________/_\________/_\________')
print('BoW            |   0.75   |   0.82\n---------------+----------+---------')
print('Bow+BoBigr     |   0.74   |   0.81\n---------------+----------+---------')
print('BoW+BoE emx    |   0.75   |   0.81\n---------------+----------+---------')
print('BoW+BoE SEL    |   0.75   |   0.81\n---------------+----------+---------')
print('Bow+BoB+BoEmx  |   0.74   |   0.81\n---------------+----------+---------')
print('Bow+BoB+BoSEL  |   0.74   |   0.81\n---------------+----------+---------')
print('Bow+BoEmx+BoSEL|   0.74   |   0.81\n---------------+----------+---------')
print('Bow+ All       |   0.73   |   0.81')

Combinación    | F1-score | Accuracy
______________/_\________/_\________
BoW            |   0.75   |   0.82
---------------+----------+---------
Bow+BoBigr     |   0.74   |   0.81
---------------+----------+---------
BoW+BoE emx    |   0.75   |   0.81
---------------+----------+---------
BoW+BoE SEL    |   0.75   |   0.81
---------------+----------+---------
Bow+BoB+BoEmx  |   0.74   |   0.81
---------------+----------+---------
Bow+BoB+BoSEL  |   0.74   |   0.81
---------------+----------+---------
Bow+BoEmx+BoSEL|   0.74   |   0.81
---------------+----------+---------
Bow+ All       |   0.73   |   0.81


## 4.2 Conclusiones

En esta tarea se experimenta sobre las posibles alternativas que se pueden tomar al pesar cada término de los documentos. Se logra notar que muchas veces es mejor normalizar las bolsas de términos para mejorar el desempeño. Se probaron además distintas variaciones en las bolsas para ver si podían servir para mejorar la bolsa de términos original. De ellos fue una bolsa de bigramas, una bolsa de emociones y una bolsa de emociones con probabilidad. Concatenando a la bolsa original **no** se logra mejorar el F1-score ni el accuraccy en ninguna combinación. El f1-score más cercano es el de BoW+BoE SEL o BoW+BoE Emx sin embargo clasifican mal 1 elemento más de la validación que la bolsa original.

Como comentario, me sorprendió que al principio el TF-IDF no hiciera tan buen trabajo y en varias ocasiones peor. También fue sorprendente todo lo que se agregó y ni así se pudo superar a la bolsa original. Una recomendación futura es que podríamos agregar un feature bueno que al evaluarlo por si solo tuviera buen desempeño puesto que todos los features nuevos que agregamos, no pasaban del .63 en accuracy, y por tanto era muy improbable lograr observar una mejora.